In [1]:
import time
import numpy as np
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
#Input examples of various QPs
X_np = np.array([[1, 1, 1, 1, 1, 2, 1, 2, 3, 4, 1, 1, 0, 1],
                [1, 1, 1, 1, 1, 2, 1, 2, 3, 4, 0, 1, 1, 1],
                [1, 1, 1, 1, 1, 2, 1, 2, 3, 4, 1, 0, 1, 1],
                [1, 1, 1, 1, 1, 2, 1, 2, 3, 4, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 4, 1, 1, 0, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 4, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 5, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 5, 0, 1, 1, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 5, 1, 1, 0, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 5, 1, 0, 1, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 6, 1, 1, 0, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 6, 1, 0, 1, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 6, 0, 1, 1, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 6, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 2, 1, 3, 2, 3, 0, 1, 1, 1],
                [1, 1, 1, 1, 1, 2, 1, 3, 2, 3, 0, 1, 1, 1],
                [1, 1, 1, 1, 1, 2, 1, 3, 2, 3, 1, 0, 1, 1],
                [1, 1, 1, 1, 1, 2, 1, 3, 2, 3, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 3, 1, 1, 0, 1],
                [1, 1, 1, 1, 1, 2, 2, 1, 3, 3, 1, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, 2, 1, 3, 3, 1, 1, 0, 1],
                [1, 1, 1, 1, 2, 2, 2, 1, 3, 3, 1, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, 2, 1, 3, 3, 0, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, 2, 1, 3, 3, 1, 0, 1, 1],
                [1, 1, 1, 1, 2, 2, -2, -3, -4, -2, 1, 1, 0, 1],
                [1, 1, 1, 1, 2, 2, -2, -3, -4, -9, 0, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, -2, -3, -4, -2, 1, 0, 1, 1],
                [1, 1, 1, 1, 2, 2, -2, -3, -4, -2, 1, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, -2, -3, -4, -3, 1, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, -2, -3, -4, -3, 1, 0, 1, 1],
                [1, 1, 1, 1, 2, 2, -2, -3, -6, -5, 1, 1, 0, 1],
                [1, 1, 1, 1, 2, 2, -2, -3, -6, -5, 0, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, -2, -3, -6, -5, 1, 0, 1, 1],
                [1, 1, 1, 1, 2, 2, -2, -3, -6, -5, 1, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, -3, -2, -6, -5, 1, 1, 0, 1],
                [1, 1, 1, 1, 2, 2, -3, -2, -6, -5, 0, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, -3, -2, -6, -5, 1, 0, 1, 1],
                [1, 1, 1, 1, 2, 2, -3, -2, -6, -5, 1, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, -3, -2, -6, -11, 1, 1, 1, 1],
                [1, 1, 1, 1, 2, 2, -3, -2, -6, -11, 1, 0, 1, 1],
                [1, 1, 2, 1, 1, 2, -3, -2, -6, -11, 1, 1, 0, 1],
                [1, 1, 2, 1, 1, 2, 2, 1, 3, 4, 0, 1, 1, 1],
                [1, 1, 2, 1, 1, 2, 2, 1, 3, 4, 1, 0, 1, 1],
                [1, 1, 2, 1, 1, 2, 2, 1, 3, 4, 1, 1, 1, 1],
                [1, 1, 2, 1, 1, 2, 3, 1, 2, 5, 1, 1, 0, 1],
                [1, 1, 2, 1, 1, 2, 3, 1, 2, 5, 0, 1, 1, 1],
                [1, 1, 2, 1, 1, 2, 3, 1, 2, 5, 1, 0, 1, 1],
                [1, 1, 2, 1, 1, 2, 3, 1, 2, 5, 1, 1, 1, 1],
                [1, 1, 2, 1, 1, 2, 1, 2, 3, 3, 1, 1, 0, 1],
                [1, 1, 2, 1, 1, 2, 1, 2, 3, 3, 1, 1, 1, 1],
                [1, 1, 2, 1, 1, 2, 3, 4, 1, 5, 1, 1, 0, 1],
                [1, 1, 2, 1, 1, 2, 3, 4, 1, 5, 0, 1, 1, 1],
                [1, 1, 2, 1, 1, 2, 3, 4, 1, 5, 1, 0, 1, 1],
                [1, 1, 2, 1, 1, 2, 3, 4, 1, 5, 1, 1, 1, 1],
                [1, 1, 2, 1, 1, 2, 4, 3, 5, 7, 1, 1, 0, 1],
                [1, 1, 2, 1, 1, 2, 4, 3, 5, 7, 0, 1, 1, 1],
                [1, 1, 2, 1, 1, 2, 4, 3, 5, 7, 1, 0, 1, 1],
                [1, 1, 2, 1, 1, 2, 4, 3, 5, 7, 1, 1, 1, 1],
                [1, 1, 2, 1, 1, 2, 2, 3, 1, 6, 1, 0, 1, 1],
                [1, 1, 2, 1, 1, 2, 2, 3, 1, 6, 1, 1, 1, 1],
                [1, 1, 2, 1, 2, 2, 2, 3, 1, 6, 1, 1, 0, 1],
                [1, 1, 2, 1, 2, 2, 2, 3, 1, 6, 0, 1, 1, 1],
                [1, 1, 2, 1, 2, 2, 2, 3, 1, 6, 1, 0, 1, 1],
                [1, 1, 2, 1, 2, 2, 2, 3, 1, 6, 1, 1, 1, 1],
                [1, 1, 2, 1, 2, 2, 1, 3, 2, 3, 1, 1, 0, 1],
                [1, 1, 2, 1, 2, 2, 1, 3, 2, 3, 0, 1, 1, 1],
                [1, 1, 2, 1, 2, 2, 1, 3, 2, 3, 1, 0, 1, 1],
                [1, 1, 2, 1, 2, 2, 1, 3, 2, 3, 1, 1, 1, 1],
                [1, 1, 2, 1, 2, 2, 1, 3, 2, 5, 1, 0, 1, 1],
                [1, 1, 2, 1, 2, 2, 1, 3, 2, 5, 0, 1, 1, 1],
                [1, 1, 2, 1, 2, 2, 2, 1, 3, 5, 1, 1, 0, 1],
                [1, 1, 2, 1, 2, 2, 2, 1, 3, 5, 0, 1, 1, 1],
                [1, 1, 2, 1, 2, 2, 2, 1, 3, 5, 1, 0, 1, 1],
                [1, 1, 2, 1, 2, 2, 2, 1, 3, 5, 1, 1, 1, 1],
                [1, 1, 2, 1, 2, 2, -2, -3, -4, -5, 1, 1, 0, 1],
                [1, 1, 2, 1, 2, 2, -2, -3, -4, -5, 0, 1, 1, 1],
                [1, 1, 2, 1, 2, 2, -2, -3, -4, -5, 1, 0, 1, 1],
                [1, 1, 2, 1, 2, 2, -2, -3, -4, -5, 1, 1, 1, 1],
                [1, 1, 2, 1, 2, 2, -3, -2, -1, -6, 1, 0, 1, 1],
                [1, 1, 2, 1, 2, 2, -3, -2, -1, -6, 1, 1, 1, 1],
                [2, 1, 1, 1, 1, 2, -3, -2, -1, -6, 1, 1, 0, 1],
                [2, 1, 1, 1, 1, 2, -3, -2, -1, -6, 1, 1, 1, 1],
                [2, 1, 1, 1, 1, 2, 3, 2, 1, 3, 1, 1, 0, 1],
                [2, 1, 1, 1, 1, 2, 3, 2, 1, 3, 0, 1, 1, 1],
                [2, 1, 1, 1, 1, 2, 3, 1, 2, 3, 1, 0, 1, 1],
                [2, 1, 1, 1, 1, 2, 3, 1, 2, 3, 1, 1, 1, 1],
                [2, 1, 1, 1, 1, 2, 2, 3, 1, 5, 1, 1, 0, 1],
                [2, 1, 1, 1, 1, 2, 2, 3, 1, 5, 0, 1, 1, 1],
                [2, 1, 1, 1, 1, 2, 2, 3, 1, 5, 1, 0, 1, 1],
                [2, 1, 1, 1, 1, 2, 2, 3, 1, 5, 1, 1, 1, 1],
                [2, 1, 1, 1, 1, 2, 1, 3, 2, 5, 1, 1, 0, 1],
                [2, 1, 1, 1, 1, 2, 1, 3, 2, 5, 0, 1, 1, 1],
                [2, 1, 1, 1, 1, 2, 1, 3, 2, 5, 1, 0, 1, 1],
                [2, 1, 1, 1, 1, 2, 1, 3, 2, 5, 1, 1, 1, 1],
                [2, 1, 1, 1, 1, 2, 1, 2, 3, 6, 1, 1, 0, 1],
                [2, 1, 1, 1, 1, 2, 1, 2, 3, 6, 0, 1, 1, 1],
                [2, 1, 1, 1, 1, 2, 1, 2, 3, 6, 1, 0, 1, 1],
                [2, 1, 1, 1, 1, 2, 1, 2, 3, 6, 1, 1, 1, 1],
                [2, 1, 1, 1, 1, 2, 3, 2, 1, 5, 1, 1, 0, 1],
                [2, 1, 1, 1, 1, 2, 3, 2, 1, 5, 1, 0, 1, 1]])

y_np = np.array([[3],
                [3],
                [3],
                [3],
                [3],
                [3],
                [5],
                [5],
                [5],
                [5],
                [7],
                [7],
                [7],
                [7],
                [2],
                [2],
                [3],
                [2],
                [3],
                [1],
                [5],
                [1],
                [1],
                [1],
                [4],
                [1],
                [4],
                [4],
                [4],
                [4],
                [4],
                [2],
                [4],
                [4],
                [4],
                [2],
                [4],
                [4],
                [4],
                [4],
                [4],
                [5],
                [5],
                [5],
                [5],
                [5],
                [5],
                [5],
                [6],
                [1],
                [6],
                [6],
                [6],
                [6],
                [6],
                [6],
                [6],
                [6],
                [7],
                [7],
                [7],
                [7],
                [7],
                [7],
                [2],
                [2],
                [6],
                [2],
                [3],
                [3],
                [5],
                [5],
                [5],
                [5],
                [4],
                [2],
                [4],
                [4],
                [4],
                [4],
                [4],
                [1],
                [4],
                [5],
                [4],
                [4],
                [6],
                [6],
                [6],
                [6],
                [3],
                [3],
                [3],
                [3],
                [7],
                [7],
                [7],
                [7],
                [6],
                [6]])

In [3]:
print "X_np shape: ", X_np.shape
print "Y_np.shape: ", y_np.shape

X_np shape:  (100, 14)
Y_np.shape:  (100, 1)


In [4]:
# N is batch size; D_in is input dimension;
# H_{i} is the dimension of the ith hidden layer; D_out is output dimension.

N, D_in, H_1, H_2, D_out = 100, 14, 25, 50, 7

In [5]:
def multi_class_format(numpy_array):
    new = []
    for item in y_np:
        for i in item:
            new.append(i)
    
    class_labels = []
    
    for i in new:
        identifier = i-1
        class_labels.append(identifier)
        
    return class_labels

Y_array = multi_class_format(y_np)
print Y_array[:25]

[2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 6, 6, 6, 6, 1, 1, 2, 1, 2, 0, 4, 0, 0, 0, 3]


In [6]:
nb_classes = 7
def get_one_hot(targets, nb_classes):
    return np.eye(nb_classes)[np.array(targets).reshape(-1)]

In [7]:
y_one_hot = get_one_hot(Y_array, 7)
print "Output shape: {} ".format(y_one_hot.shape)
print y_one_hot

Output shape: (100, 7) 
[[ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  

In [8]:
# Make a train/test split using 20% test size
X_train, X_test, y_train, y_test = train_test_split(X_np, y_one_hot, test_size=0.20)
print "X_train: {}, y_train: {} , X_test: {} , y_test: {} ".format(X_train.shape[0], y_train.shape[0], X_test.shape[0], y_test.shape[0])

X_train: 80, y_train: 80 , X_test: 20 , y_test: 20 


In [9]:
X_train = torch.from_numpy(X_train).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)

X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

print "Training set shape: {} ".format(X_train.size())
print "Training labels shape: {} ".format(y_train.size())

print "Test set shape: {} ".format(X_test.size())
print "Test labels shape: {} ".format(y_test.size())

Training set shape: torch.Size([80, 14]) 
Training labels shape: torch.Size([80, 7]) 
Test set shape: torch.Size([20, 14]) 
Test labels shape: torch.Size([20, 7]) 


In [10]:
net = torch.nn.Sequential(
    torch.nn.Linear(D_in, H_1),
    torch.nn.ReLU(),
    torch.nn.Linear(H_1, H_2),
    torch.nn.ReLU(),
    torch.nn.Linear(H_2, D_out)
    )

In [11]:
#net = Net()
print net

Sequential(
  (0): Linear(in_features=14, out_features=25)
  (1): ReLU()
  (2): Linear(in_features=25, out_features=50)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=7)
)


In [12]:
#Converting the training one hot vector labels into indices because PyTorch does not take one-hot vectors
targetnp = y_train.numpy()
idxs = np.where(targetnp > 0)[1]
new_targets = torch.LongTensor(idxs)

In [13]:
#Converting the test one hot vector labels into indices
test_targetnp = y_test.numpy()
test_idxs = np.where(test_targetnp > 0)[1]
new_test_targets = torch.LongTensor(test_idxs)

In [14]:
# create a Adam optimizer
learning_rate = 1e-3
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay = 1e-2)
# create a loss function
loss_func = nn.CrossEntropyLoss()

In [15]:
# run the main training loop
epochs = 2000
train_loss = 0
train_correct = 0
test_correct = 0
for epoch in range(epochs):
        data, target = Variable(X_train), Variable(new_targets)
        optimizer.zero_grad()
        net_out = net(data)
        #print net_out
        #print target.data
        
        #sum up the loss
        train_loss = loss_func(net_out, target)
        pred = net_out.data.max(1)[1] #get the index of the max log probabitlity
        train_correct = pred.eq(target.data).sum()
        
        train_loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print('\nTraining Epoch: {} Average loss: {:.4f}, Training Accuracy: {}/{} ({:.4f}%)\n'.format(
        epoch, train_loss.data[0], train_correct, len(target),
        100. * train_correct / len(target)))



Training Epoch: 0 Average loss: 2.0097, Training Accuracy: 6/80 (7.5000%)


Training Epoch: 100 Average loss: 1.1309, Training Accuracy: 54/80 (67.5000%)


Training Epoch: 200 Average loss: 0.5364, Training Accuracy: 64/80 (80.0000%)


Training Epoch: 300 Average loss: 0.2915, Training Accuracy: 74/80 (92.5000%)


Training Epoch: 400 Average loss: 0.1939, Training Accuracy: 77/80 (96.2500%)


Training Epoch: 500 Average loss: 0.1326, Training Accuracy: 80/80 (100.0000%)


Training Epoch: 600 Average loss: 0.1089, Training Accuracy: 80/80 (100.0000%)


Training Epoch: 700 Average loss: 0.0988, Training Accuracy: 80/80 (100.0000%)


Training Epoch: 800 Average loss: 0.0935, Training Accuracy: 80/80 (100.0000%)


Training Epoch: 900 Average loss: 0.0905, Training Accuracy: 80/80 (100.0000%)


Training Epoch: 1000 Average loss: 0.0884, Training Accuracy: 80/80 (100.0000%)


Training Epoch: 1100 Average loss: 0.0870, Training Accuracy: 80/80 (100.0000%)


Training Epoch: 1200 Average loss:

In [16]:
test_loss = 0
test_correct = 0

#Converting test data and target labels into pyTorch variables
test_data, test_target = Variable(X_test), Variable(new_test_targets)

tic = time.time()
#Test set output
net_out = net(test_data)

test_loss = loss_func(net_out, test_target)
test_pred = net_out.data.max(1)[1] #get the index of max log probability
test_correct = test_pred.eq(test_target.data).sum()

toc = time.time()
time_diff = toc - tic

print('\nTest loss: {:.4f}, Test accuracy: {}/{} ({:.4f}%)\n'.format(
    test_loss.data[0], test_correct, len(test_target), 100. * test_correct / len(test_target)))

print "Time taken to solve 20 problems: ", time_diff


Test loss: 0.2390, Test accuracy: 19/20 (95.0000%)

Time taken to solve 20 problems:  0.000648021697998
